In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt

In [2]:
TRAINING_NAME = 'loss_weights_balanced'

ckpt_folder = './ckpt/'+ TRAINING_NAME +'/'

global_step = tf.get_variable(
                    'global_step', [],
                    initializer=tf.constant_initializer(0), trainable=False)

alternate_global_step = tf.placeholder(tf.int32)

MAX_ITERATIONS = 200000
learning_rate = tf.train.polynomial_decay(0.0001, alternate_global_step,
                                          MAX_ITERATIONS, 0.000001,
                                          power=3)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [3]:
# import mnist
mnist = input_data.read_data_sets("MNIST_data/")
train_X = mnist.train.images
test_X = mnist.test.images

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
batch_size = 4
batch_train_x = mnist.train.next_batch(batch_size)
batch_test_x = mnist.test.next_batch(batch_size)
imgs_train= batch_train_x[0].reshape((-1, 28, 28, 1))                               
imgs_test = batch_test_x[0].reshape((-1, 28, 28, 1))

In [7]:
train_tensor = tf.convert_to_tensor(imgs_train)

conv1 = tf.layers.conv2d(inputs=train_tensor, filters=16, kernel_size=2, strides=2, activation=tf.nn.leaky_relu, name='conv1')
conv2 = tf.layers.conv2d(inputs=conv1, filters=32, kernel_size=2, strides=2, activation=tf.nn.leaky_relu, name='conv2')

flattened = tf.layers.flatten(conv2)
latent_space = tf.layers.dense(flattened,200, name='latent_space')
reshaped_flattened = tf.layers.dense(latent_space,1568, name='reshaped_flattened')
reshaped = tf.reshape(reshaped_flattened,[4,7,7,32])


convt_1 = tf.layers.conv2d_transpose(reshaped, filters=32, kernel_size=2, strides=2, activation=tf.nn.leaky_relu, name='convt_1')
convt_2 = tf.layers.conv2d_transpose(convt_1, filters=16, kernel_size=2, strides=2, activation=tf.nn.leaky_relu, name='convt_2')
convt_3 = tf.layers.conv2d_transpose(convt_2, filters=3, kernel_size=1, strides=1, activation=tf.nn.tanh, name='convt_3')

ValueError: Variable conv1/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-6-c3cee4fa99f4>", line 3, in <module>
    conv1 = tf.layers.conv2d(inputs=train_tensor, filters=16, kernel_size=2, strides=2, activation=tf.nn.leaky_relu, name='conv1')
  File "/home/muazzam/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/muazzam/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3185, in run_ast_nodes
    if (yield from self.run_code(code, result)):


In [9]:
with tf.variable_scope('reconstruction_loss_l2'):
    loss = tf.reduce_mean(tf.square(convt_3 - train_tensor))
    loss = tf.losses.compute_weighted_loss(loss, weights=1200)

In [10]:
opt = tf.train.AdamOptimizer(learning_rate, beta1=0.5, beta2=0.999)
grads = opt.compute_gradients(loss)
apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)

variable_averages = tf.train.ExponentialMovingAverage(0.9999, global_step)
variables_averages_op = variable_averages.apply(tf.trainable_variables())
train_op = tf.group(apply_gradient_op, variables_averages_op)